In [1]:
from sqlmodel import SQLModel, Relationship, Field, create_engine
from pydantic import EmailStr
from typing import Optional, List  # Added List import
from pydantic_extra_types.phone_numbers import PhoneNumber
from datetime import datetime
from enum import Enum


class User(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    username: str = Field(unique=True)
    name: str = Field(max_length=50)
    surname: str = Field(max_length=50)
    email: EmailStr
    cell: PhoneNumber
    password_hash: str
    coin_balance: Optional[float] = None
    role: Optional[str] = Field(default="user")

    # Relationships
    bank_account: Optional["BankAccount"] = Relationship(back_populates="owner")
    
    referrals_made: List["Referral"] = Relationship(
        back_populates="referrer",
        sa_relationship_kwargs={"foreign_keys": "[Referral.referrer_id]"},
    )
    referral_record: Optional["Referral"] = Relationship(
        back_populates="referred",
        sa_relationship_kwargs={"foreign_keys": "[Referral.referred_id]"},
    )
    
    # Fixed: Added listings relationship
    listings: List["Listing"] = Relationship(back_populates="seller")
    
    transactions_bought: List["Transaction"] = Relationship(
        back_populates="buyer", 
        sa_relationship_kwargs={"foreign_keys": "[Transaction.buyer_id]"}
    )
    transactions_sold: List["Transaction"] = Relationship(
        back_populates="seller", 
        sa_relationship_kwargs={"foreign_keys": "[Transaction.seller_id]"}
    )


class BankAccount(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(max_length=20)
    number: int
    branch_code: int = Field(ge=0, le=99999)  # Fixed: ge (>=) and le (<=) instead of lt/gt

    user_id: Optional[int] = Field(default=None, foreign_key="user.id")
    owner: Optional[User] = Relationship(back_populates="bank_account")


class Referral(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    referrer_id: int = Field(foreign_key="user.id")
    referred_id: int = Field(foreign_key="user.id")
    date: datetime

    referrer: Optional["User"] = Relationship(
        back_populates="referrals_made",
        sa_relationship_kwargs={"foreign_keys": "[Referral.referrer_id]"},
    )
    referred: Optional["User"] = Relationship(
        back_populates="referral_record",
        sa_relationship_kwargs={"foreign_keys": "[Referral.referred_id]"},
    )


class AuctionStatus(Enum):
    UPCOMING = "UPCOMING"  # Fixed: spelling and consistency
    STARTED = "STARTED"
    ENDED = "ENDED"


class Auction(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    start_time: datetime  # Added space for consistency
    end_time: datetime
    status: AuctionStatus = Field(default=AuctionStatus.UPCOMING)
    
    # Fixed: lowercase back_populates and List type
    listings: List["Listing"] = Relationship(back_populates="auction")


class Listing(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    amount: float
    seller_id: int = Field(foreign_key="user.id")
    date: datetime
    auction_id: int = Field(foreign_key="auction.id")

    # Relationships
    auction: Optional[Auction] = Relationship(back_populates="listings")
    # Fixed: should point to seller's listings, not transactions_sold
    seller: Optional[User] = Relationship(back_populates="listings")
    transaction: Optional["Transaction"] = Relationship(back_populates="listing")


class Transaction(SQLModel, table=True):  # Fixed: class name spelling
    id: Optional[int] = Field(default=None, primary_key=True)
    buyer_id: int = Field(foreign_key="user.id")
    seller_id: int = Field(foreign_key="user.id")
    status: str
    date: datetime
    listing_id: int = Field(foreign_key="listing.id", unique=True)  # Unique constraint for 1:1

    # Relationships
    listing: Optional[Listing] = Relationship(back_populates="transaction")
    buyer: Optional[User] = Relationship(
        back_populates="transactions_bought", 
        sa_relationship_kwargs={"foreign_keys": "[Transaction.buyer_id]"}
    )
    seller: Optional[User] = Relationship(
        back_populates="transactions_sold", 
        sa_relationship_kwargs={"foreign_keys": "[Transaction.seller_id]"}
    )

In [2]:
db_name = "auction.db"
db_url = f"sqlite:///{db_name}"
engine = create_engine(db_url, echo=True)


def create_db_and_tables():
    SQLModel.metadata.create_all(engine)


In [3]:
create_db_and_tables()

2025-06-10 00:17:08,826 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-10 00:17:08,827 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2025-06-10 00:17:08,828 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-10 00:17:08,829 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2025-06-10 00:17:08,830 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-10 00:17:08,830 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bankaccount")
2025-06-10 00:17:08,831 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-10 00:17:08,832 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("bankaccount")
2025-06-10 00:17:08,833 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-10 00:17:08,834 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("referral")
2025-06-10 00:17:08,834 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-10 00:17:08,836 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("referral")
2025-06-10 00:17:08,836 INFO sqlalchemy.engine.Engine [raw sql